# Road Following 
このnotebookdでは、カメラ画像を読み込み、JetBotの進行方向となるターゲットポイントをクリックして、画像の座標となるx、y値を取得します。画像とx、y値を学習用のデータセットとして保存します。

## パッケージの更新
新しくなったJetBotの機能を使うために、一度ターミナルからパッケージの更新が必要となります。まだ更新していない場合は、下記の手順で更新を実行してください。  
ターミナルは、JupyterのTerminalを起動するか、sshでJetBotにログインするか、キーボード+マウス+モニターをJetBotに接続してCtrl+ALT+Tでターミナルを開くことができます。

* MAX-N mode:ビルド時間を短くするために、Jetsonの最高性能を出すためのモードに変更します。
> sudo nvpmodel -m 0

* nodejs:マウスクリックウィジェットで必要になります。
> sudo python3 -m pip install git+https://github.com/ipython/traitlets@dead2b8cdde  
sudo apt-get install -y curl  
curl -sL https://deb.nodesource.com/setup_10.x | sudo -E bash -  
sudo apt-get install -y nodejs libffi-dev  

* jupyter_clickable_image_widget:マウスクリックウィジェット
> cd
git clone https://github.com/jaybdub/jupyter_clickable_image_widget  
cd jupyter_clickable_image_widget  
sudo -H pip3 install -e .  
sudo jupyter labextension install js  

* bokeh:グラフ表示ウィジェット
> sudo apt-get install -y python3-matplotlib  
sudo -H pip3 install bokeh  
sudo jupyter labextension install @bokeh/jupyter_bokeh  

* 5W mode:5V3AのモバイルバッテリーではMAX-Nモードの安定動作は電流的に厳しいので、5Wモードに変更します。
> sudo nvpmodel -m 1

* reboot:再起動します。モバイルバッテリーでMAX-Nモードのまま再起動すると起動しないことがあります。その場合は5VのACアダプターを使って起動後、5Wモードに変更してください。
> sudo reboot

## それでは、始めましょう
collision avoidanceを実行していれば、今回実行する次の3ステップには馴染みがあるでしょう。

1. データ収集
2. 学習
3. 走行

collision avoidanceでは、画像に対して2つのラベル（フリー、ブロック）のどちらに該当するかを判断することができました。画像を2クラスに分類しているため、これを画像分類と呼びます。  
road followingでも、データ収集から走行までは同じような流れで実行していきます。  
ただし、今回はJetBotが道路（または任意のパスや通路、コース）を走行できるようにするために、画像分類の代わりに、**回帰**という別の手法で学習させます。

 1. JetBotを道路上のさまざまな位置に配置します（中心からのオフセット、さまざまな角度など）

> データの多様性こそが重要であると、collision avoidanceのときに覚えたはずです。


2. JetBotからのライブカメラの画像を画面に表示します
3. マウスでX,Y座標を移動することで、`greenの点`をロボットが移動したい方向に移動します。
4. この`greenの点`のX、Y座標の値をロボットのカメラからの画像と、一緒に保存します


`train_model.ipynb`による学習では、集めたデータを基にX, Yの値の推論をおこなうための学習をします。`live_demo.ipynb`によるライブデモでは、JetBotのステアリングの値を計算するために予測されたX、Y値を使用します。(角度は「完全に正確」ではありません。本来であればカメラレンズの歪みを補正するための画像キャリブレーションが必要になりますが、JetBotの動作は角度にほぼ比例するため、キャリブレーション無しでもJetBotの制御は正常に機能します)

それでは、この`data_collection.ipynb`の重要な点は何でしょうか？ 下記が役立つと思われるガイドです。

1. カメラからのライブ映像を見ます
2. JetBotがたどるべき経路を想像してください（道路からの脱輪などを回避するために必要な距離を計算してみてください）
3. JetBotが道路をふらつかずにまっすぐ進むことができるように、目標点はこの経路に沿ったできるだけ遠くの位置に配置します。

> 例として、もし真っ直ぐの道路があったとします、目標点は地平線にします。もし道路が鋭く曲がったカーブなら、脱輪しないレベルでJetBotの近くに配置する必要があるでしょう。
 
ディープラーニングのモデルを意図したとおりに機能させるために、十分なデータ量を取得します。  
意図したとおりに動作するモデルでは、以下の動作が期待できます。

1. JetBotは、目標点に向かって安定して移動できます（道路の外に出ることなく）。
2. 経路に沿って設定した目標点を目指して継続的に処理されます。

## Labeling example video(ラベリングのデモ動画)

下記URLの動画を参考に、画像にラベルを付ける方法の例を確認します。このモデルはたった123枚の画像で動作しました:)

YouTubeのサンプル動画：[https://www.youtube.com/embed/FW4En6LejhI](https://www.youtube.com/embed/FW4En6LejhI)

## Import Libraries(ライブラリの読み込み)

In [ ]:
# IPython Libraries for display and widgets
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import glob
import numpy as np
import cv2
import time

## Create Live Camera Widget(ライブカメラウィジェットの作成)

本サンプルを実行するにあたり、まずnvargus-daemon(カメラ等で使用)をリスタートします。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

カメラの初期化と表示をおこないます。

JetBotのカメラクラスは、CSI MIPI cameraを有効にするために使います。road followingに使うニューラルネットワークのモデルでは、224x224ピクセルの画像データを入力として使います。画像サイズが大きくなると、ニューラルネットワークモデルの学習や実行に必要とするメモリ量と処理時間が大幅に増えます。Jetson Nanoでは最適な値としてこのサイズを設定します。

In [ ]:
camera = Camera.instance(width=224, height=224, fps=2)

## Data Collection(データ収集)

カメラ画像とカメラ画像にターゲットを追加したウィジェットを作成します。今回は画像をクリックしてデータ注釈の座標を取得できるようにする[jupyter_clickable_image_widget](https://github.com/jaybdub/jupyter_clickable_image_widget)と呼ばれる特別なipywidgetを使用します。これによりカメラ画面をクリックするだけでデータ収集が可能になります。

カメラ画像を保存する機能を作成します。

1. 目標点をクリックします。

カメラ画面をクリックすると、カメラ画像をターゲットの座標x,yとuuidをファイル名に含めて保存します。  
これは``dataset_xy``フォルダーに、  
``xy_<x value>_<y value>_<uuid>.jpg``  
の形式で保存されます。

学習時に、ファイル名からx,yの値を復元し、イメージとともに読み込みます。

実行時に`jupyter lab build`するようにポップアップが表示されることがあります。  
これはターミナルで`sudo jupyter lab build`を実行すると解決します。

In [ ]:
from jupyter_clickable_image_widget import ClickableImageWidget
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
camera_link = traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
count_widget = ipywidgets.IntText(description='count')
# manually update counts at initialization
count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

# カメラ画像を保存する機能を作成します。
def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        #dataset.save_entry(category_widget.value, camera.value, x, y)
        x_ratio = int((x/224)*100)
        y_ratio = int((y/224)*100)
        uuid = 'xy_%03d_%03d_%s' % (x_ratio, y_ratio, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    count_widget
])

# 操作ボタンを作成

画像データを集めやすくするために、basic motionと同じようにボタン操作でJetBotを操作出来るように準備します。  
ボタンは後ほどカメラ画面と一緒に表示します。

In [ ]:
robot = Robot()

def stop(change):
    robot.stop()

# カーペットの上だと旋回出来ないことがあるので、少し出力を上げます。
def step_forward(change):
    robot.forward(0.6)
    time.sleep(0.3)
    robot.stop()

def step_backward(change):
    robot.backward(0.6)
    time.sleep(0.3)
    robot.stop()

def step_left(change):
    robot.left(0.6)
    time.sleep(0.1)
    robot.stop()

def step_right(change):
    robot.right(0.6)
    time.sleep(0.1)
    robot.stop()

# create control buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

# 画面と操作ボタンを表示して始めよう

次のコードを実行すると、画面と操作ボタンが表示されます。  
左の画面でJetBotが目標とする場所をクリックすると、データが追加されます。クリックした箇所は右の画面に緑色のまるで表示されます。  
JetBotを操作しながら、100枚程度のデータがあればうまく学習できると思います。

In [ ]:
# control buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])

display(widgets.HBox([
    data_collection_widget,
    # display buttons
    widgets.VBox([
        controls_box
    ])
]))

# カメラ停止
最後に、他のノートブックでカメラを使うために、このノートブックで使ったカメラを停止しておきます。

In [ ]:
camera_link.unlink()  # ブラウザへのストリーミングを停止します（カメラは引き続き実行されています）
camera.stop()

## Next(次)

次は、``train_model.ipynb``で学習をおこないます。  
ノートブックメニューから`Kernel`->`Restert Kernel`を選んでJupyter kernelを再起動するか、JetBotを一度再起動してから次に進むとスムーズに進行できます。

[train_model.ipynb](./train_model.ipynb) をクリックし、移動します。